# E2B Code Sandbox with Google Gemini
Execute AI-generated Python code in a secure sandbox environment.

## Setup and Imports

In [2]:
"""E2B Code Sandbox with Google Gemini."""
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import base64
import pandas as pd
import time

root_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root_dir)

from dotenv import load_dotenv
load_dotenv()

from e2b_code_interpreter import Sandbox
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

## Initialize Model and Sandbox

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

checkpointer = InMemorySaver()

sbx = Sandbox.create()
print("Sandbox created")

Sandbox created


## Helper Functions

In [4]:
def get_dataset_info(file_path):
    """Get basic dataset info."""
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, nrows=3)
    else:
        df = pd.read_excel(file_path, nrows=3)

    return f"Columns: {list(df.columns)}\nSample data:\n{df.to_string()}"

## Define Tools

In [5]:
@tool
def upload_file(local_file_path: str):
    """Upload a data file to the E2B sandbox for analysis.
    
    Use this tool to upload CSV or Excel files before analyzing them.
    The file will be uploaded to the sandbox and the sandbox path will be returned.
    
    Args:
        local_file_path: Local path to the file (e.g., "./data/IMDB-Movie-Data.csv")
        
    Returns:
        Success message with sandbox_path and dataset_info
        
    Example:
        local_file_path = "./data/titanic.csv"
    """
    if not os.path.exists(local_file_path):
        return f"Error: File not found at {local_file_path}"
    
    filename = os.path.basename(local_file_path)
    
    with open(local_file_path, "rb") as f:
        sandbox_file = sbx.files.write(f"data/{filename}", f)
    
    dataset_info = get_dataset_info(local_file_path)
    
    return f"File uploaded successfully!\nSandbox path: {sandbox_file.path}\n{dataset_info}"

@tool
def run_python_code(code: str):
    """Execute Python code in E2B sandbox and save chart outputs.
    
    IMPORTANT: The code parameter must be valid, executable Python code only.
    - Do NOT include markdown formatting (no ```python or ```)
    - Do NOT include explanations or comments outside the code
    - Include all necessary imports (pandas, matplotlib, numpy, etc.)
    - Use the exact dataset path provided in the system prompt
    - For visualizations, end with: display(plt.gcf())

    Args:
        code: Valid executable Python code as a plain string

    Returns:
        Execution result or error message
        
    Example:
        code = "import pandas as pd\\nimport matplotlib.pyplot as plt\\ndf = pd.read_csv('/path/to/data.csv')\\nplt.plot(df['x'], df['y'])\\ndisplay(plt.gcf())"
    """
    print('Running code in sandbox....')
    execution = sbx.run_code(code)
    print('Code execution finished!')

    if execution.error:
        return f"Error: {execution.error.name}\nValue: {execution.error.value}"

    os.makedirs('images', exist_ok=True)
    
    results = []
    timestamp = int(time.time())
    for idx, result in enumerate(execution.results):
        if result.png:
            filename = f'images/{timestamp}_chart-{idx}.png'
            with open(filename, 'wb') as f:
                f.write(base64.b64decode(result.png))
            results.append(f'Chart saved to {filename}')

    return "\n".join(results) if results else "Code executed successfully"

## Run Analysis

## Create Agent and Execute Query

In [ ]:
system_prompt = """You are a data analysis assistant with access to filesystem search, file upload, and Python code execution tools.

WORKFLOW:
1. First, use filesystem search tools to locate the requested data file in the local filesystem
2. Once you find the file path, use the upload_file tool to upload it to the sandbox
3. The upload_file tool will return the sandbox path and dataset information
4. Finally, use run_python_code tool to analyze the data using the sandbox path

CRITICAL RULES for run_python_code:
1. Generate ONLY executable Python code - no explanations, no markdown, no comments outside code
2. Import all required libraries (pandas, matplotlib, numpy, etc.)
3. Load data from the sandbox path returned by upload_file (e.g., /home/user/data/filename.csv)
4. For plots, MUST end with: display(plt.gcf())
5. Use the run_python_code tool with ONLY the code string

Example workflow:
- User asks: "Analyze IMDB movie data"
- You search filesystem for IMDB related files
- You find "./data/IMDB-Movie-Data.csv"
- You upload it using upload_file tool
- You get sandbox path like "/home/user/data/IMDB-Movie-Data.csv"
- You generate Python code using that exact sandbox path"""



Running code in sandbox....
Code execution finished!

Response:
I have created a line chart showing the average movie ratings over the years.


In [ ]:
from langchain.agents.middleware import FilesystemFileSearchMiddleware

agent = create_agent(
    model=model,
    tools=[upload_file, run_python_code],
    system_prompt=system_prompt,
    checkpointer=checkpointer,
    middleware=[FilesystemFileSearchMiddleware(
        root_path="./data",
        use_ripgrep=True,
        max_file_size_mb=100
    )]
)

config = {"configurable": {"thread_id": "default"}}

query = "Upload ./data/IMDB-Movie-Data.csv and create a line chart showing average ratings over years"

result = agent.invoke({"messages": [HumanMessage(content=query)]}, config=config)

response = result['messages'][-1].text
print(f"\nResponse:\n{response}")


Response:
I am unable to upload the file `/IMDB-Movie-Data.csv` as the `upload_file` tool reports that the file is not found, even though `glob_search` indicates its presence. This prevents me from proceeding with the analysis.
